<a href="https://colab.research.google.com/github/nyannnyan/ColabData/blob/main/NNLab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.neural_network import MLPClassifier

# ANDの入力と出力
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
y = [0, 0, 0, 1]

# 隠れ層なし、線形活性化関数、lbfgsソルバー
clf = MLPClassifier(hidden_layer_sizes=(), activation='identity', solver='lbfgs')
clf.fit(X, y)

# 予測を確認
print(clf.predict(X))

[0 0 0 1]


In [2]:
# ORの入力と出力
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
y = [0, 1, 1, 1]

# 隠れ層なし、線形活性化関数、lbfgsソルバー
clf = MLPClassifier(hidden_layer_sizes=(), activation='identity', solver='lbfgs')
clf.fit(X, y)

# 予測を確認
print(clf.predict(X))

[0 1 1 1]


In [3]:
# XORの入力と出力
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
y = [0, 1, 1, 0]

# 隠れ層なし、線形活性化関数、lbfgsソルバー
clf = MLPClassifier(hidden_layer_sizes=(), activation='identity', solver='lbfgs')
clf.fit(X, y)

# 予測を確認
print(clf.predict(X))

[1 1 0 0]


In [4]:
clf = MLPClassifier(hidden_layer_sizes=(4, 2), activation='identity', solver='lbfgs')
clf.fit(X, y)

print(clf.predict(X))

[0 0 0 1]


In [5]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(4, 2), activation='tanh', solver='lbfgs', max_iter=1000)

for i in range(10):
    clf.fit(X, y)
    print(f"試行{i}: 予測値 = {clf.predict(X)}")

試行0: 予測値 = [0 1 1 0]
試行1: 予測値 = [0 1 1 0]
試行2: 予測値 = [0 1 1 0]
試行3: 予測値 = [0 1 1 0]
試行4: 予測値 = [0 1 1 0]
試行5: 予測値 = [0 1 1 0]
試行6: 予測値 = [0 1 1 0]
試行7: 予測値 = [0 1 1 0]
試行8: 予測値 = [0 1 1 0]
試行9: 予測値 = [0 1 1 0]


In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

dataset = load_digits()
X = dataset.data
y = dataset.target

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

# 隠れ層2つ(50ノード、20ノード)、relu活性化関数、adamソルバー
clf = MLPClassifier(hidden_layer_sizes=(50, 20), activation='relu', solver='adam', max_iter=500)
clf.fit(train_X, train_y)

test_y_pred = clf.predict(test_X)
acc = accuracy_score(test_y, test_y_pred)
print(f"テストデータに対する正解率: {acc*100:.2f}%")

テストデータに対する正解率: 96.11%


In [8]:
!pip install torch torchvision  # Install PyTorch and torchvision if not already installed


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [10]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # Input channels: 1 (grayscale), Output channels: 6, Kernel size: 5
        self.pool = nn.MaxPool2d(2, 2)  # Pooling size: 2x2
        self.conv2 = nn.Conv2d(6, 16, 5)  # Input channels: 6, Output channels: 16, Kernel size: 5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # Fully connected layer 1, Input features: 16 * 4 * 4 (from conv2 output), Output features: 120
        self.fc2 = nn.Linear(120, 84)  # Fully connected layer 2, Input features: 120, Output features: 84
        self.fc3 = nn.Linear(84, 10)  # Fully connected layer 3 (output layer), Input features: 84, Output features: 10 (number of classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # ReLU activation after conv1, then pooling
        x = self.pool(F.relu(self.conv2(x)))  # ReLU activation after conv2, then pooling
        x = x.view(-1, 16 * 4 * 4)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))  # ReLU activation for fc1
        x = F.relu(self.fc2(x))  # ReLU activation for fc2
        x = self.fc3(x)  # Output layer without activation (assuming softmax in loss function)
        return x


In [11]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 10

# Download and transform MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the model, optimizer, and loss function
model = LeNet5()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
